<a href="https://colab.research.google.com/github/simicvm/simple-stable-diffusion/blob/main/stable-diffusion-inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Best Avaliable Stable Diffusion™
Use the [v1-5 model released by Runwayml](https://huggingface.co/runwayml/stable-diffusion-v1-5) together with the [fine-tuned VAE decoder by StabilityAI](https://huggingface.co/stabilityai/sd-vae-ft-mse)

In [ ]:
#@title Install Stuff
!pip install diffusers transformers ftfy huggingface_hub
from PIL import Image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
#@title Login to the Hugging Face Hub
#@markdown Don't forget to visit [the repo page](https://huggingface.co/runwayml/stable-diffusion-v1-5), read and accept the LICENSE if you agree
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#@title Set up the pipeline
from diffusers.models import AutoencoderKL
from diffusers import StableDiffusionPipeline
from torch import autocast
import torch

model = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}
vae = "sd-vae-ft-mse" #@param["original", "sd-vae-ft-mse", "sd-vae-ft-ema"]
if(vae!="original"):
  vae = AutoencoderKL.from_pretrained(f"stabilityai/{vae}")
  pipe = StableDiffusionPipeline.from_pretrained(model, vae=vae, torch_dtype=torch.float16, revision="fp16")
else:
  pipe = StableDiffusionPipeline.from_pretrained(model, torch_dtype=torch.float16, revision="fp16") 
pipe.to("cuda")

In [ ]:
#@title Run the Stable Diffusion pipeline
#@markdown Don't forget to use the placeholder token in your prompt

prompt = "a delicious popcorn cheesecake slice" #@param {type:"string"}
num_samples = 2 #@param {type:"number"}
seed = -1 #@param {type:"integer"}
steps = 50
guidance_scale = 7.5
with autocast("cuda"):
  if(seed == -1):
    images = pipe([prompt] * num_samples, num_inference_steps=steps, guidance_scale=guidance_scale).images
  else:
    generator = torch.Generator("cuda").manual_seed(seed)
    images = pipe([prompt] * num_samples, num_inference_steps=steps, guidance_scale=guidance_scale, generator=generator).images
grid = image_grid(images, 1, num_samples)
grid